# Homework 01

**1**. (25 points)

The code below gives five "documents" with titles in `titles` and text in `contents`. 

- Convert each text into "words" by converting to lower case, removing punctuation and splitting on whitespace
- Make a list of all unique "words" in any of the texts
- Create an pandas DataFrame whose rows are words, columns are titles, and values are counts of the word in the document
- Add a column `total` that counts the total number of occurrences for each word across all documents
- Show the rows for the 5 most commonly used words

In [4]:
import sklearn
from sklearn.datasets import fetch_20newsgroups
twenty = fetch_20newsgroups(subset='train')
target_names = twenty['target_names']
titles = [target_names[i] for i in twenty['target'][2:7]]
contents = twenty['data'][2:7]

In [5]:
import string
texts = []
for i in range(len(contents)):
    c = contents[i].lower()
    texts.append(c.translate(str.maketrans('', '', string.punctuation))) # remove punctuation
#     texts.append(c.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))) # substitute each punctuation with white space
# print(texts[1]) # 'texts' stores texts in lower case removing punctuation

In [6]:
wordsList = [text.split() for text in texts] # list of list of words in each document
vocab = list(set([word for words in wordsList for word in words])) # list of all unique words in any of the texts
# vocab

In [7]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.zeros((len(vocab), len(titles)), dtype = int), index = vocab, columns = titles)
for doc_idx in range(len(wordsList)):
    for word in wordsList[doc_idx]:
        df[titles[doc_idx]][word] += 1

total = df.sum(axis=1)
df['total'] = total
# print(df)
total_np = np.array(total)
total_sorted_idx = list(np.argsort(total_np)[::-1][:5])
df.loc[[vocab[idx] for idx in total_sorted_idx], :]

,comp.sys.mac.hardware,comp.graphics,sci.space,talk.politics.guns,sci.med,total
the,21,4,7,17,0,49
of,6,1,1,17,3,28
a,12,1,3,2,2,20
to,5,1,1,10,2,19
i,8,0,1,6,4,19


**2**. (75 points)

A Caesar cipher is a very simple method of encoding and decoding data. The cipher simply replaces characters with the character offset by $k$ places. For example, if the offset is 3, we replace `a` with `d`, `b` with `e` etc. The cipher wraps around so we replace `y` with `b`, `z` with `c` and so on. Punctuation, spaces and numbers are left unchanged.

- Write a function `encode` that takes as arguments a string and an integer offset and returns the encoded cipher.
- Write a function `decode` that takes as arguments a cipher and an integer offset and returns the decoded string. 
- Write a function `auto_decode` that takes as argument a cipher and uses a statistical method to guess the optimal offset to decode the cipher, assuming the original string is in English which has the following letter frequency:

```python
freq = {
 'a': 0.08167,
 'b': 0.01492,
 'c': 0.02782,
 'd': 0.04253,
 'e': 0.12702,
 'f': 0.02228,
 'g': 0.02015,
 'h': 0.06094,
 'i': 0.06966,
 'j': 0.00153,
 'k': 0.00772,
 'l': 0.04025,
 'm': 0.02406,
 'n': 0.06749,
 'o': 0.07507,
 'p': 0.01929,
 'q': 0.00095,
 'r': 0.05987,
 's': 0.06327,
 't': 0.09056,
 'u': 0.02758,
 'v': 0.00978,
 'w': 0.0236,
 'x': 0.0015,
 'y': 0.01974,
 'z': 0.00074
}
```

- Encode the following nursery rhyme using a random offset from 10 to 20, then recover the original using `auto_decode`:

```text
Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;
One for the master,
And one for the dame,
And one for the little boy
Who lives down the lane.
```

In [8]:
def encode(input_str, offset):
    input_str = input_str.lower()
    output_cipher = ''
    lower_dict = dict(zip(string.ascii_lowercase, range(26))) # alphabet as key, index as value
    for i in range(len(input_str)):
        if input_str[i] in string.ascii_lowercase:
            offset_idx = lower_dict[input_str[i]] + offset
            offset_idx -= int(offset_idx/26) * 26 # maintain the index in range [-25:25] to ensure valid
            output_cipher += string.ascii_lowercase[offset_idx]
        else:
            output_cipher += input_str[i]
    return output_cipher

In [9]:
def decode(input_cipher, offset):
    output_str = ''
    lower_dict = dict(zip(string.ascii_lowercase, range(26))) # alphabet as key, index as value
    for i in range(len(input_cipher)):
        if input_cipher[i] in string.ascii_lowercase:
            offset_idx = lower_dict[input_cipher[i]] - offset
            offset_idx -= int(offset_idx/26) * 26 # maintain the index in range [-25:25] to ensure valid
            output_str += string.ascii_lowercase[offset_idx]
        else:
            output_str += input_cipher[i]
    return output_str

In [10]:
freq = {
 'a': 0.08167,
 'b': 0.01492,
 'c': 0.02782,
 'd': 0.04253,
 'e': 0.12702,
 'f': 0.02228,
 'g': 0.02015,
 'h': 0.06094,
 'i': 0.06966,
 'j': 0.00153,
 'k': 0.00772,
 'l': 0.04025,
 'm': 0.02406,
 'n': 0.06749,
 'o': 0.07507,
 'p': 0.01929,
 'q': 0.00095,
 'r': 0.05987,
 's': 0.06327,
 't': 0.09056,
 'u': 0.02758,
 'v': 0.00978,
 'w': 0.0236,
 'x': 0.0015,
 'y': 0.01974,
 'z': 0.00074
}

In [83]:
def auto_decode(input_cipher):
    optimal_offset = 0
    optimal_str = ''
    count_list = [0] * 26
    lower_dict = dict(zip(string.ascii_lowercase, range(26)))
    for char in input_cipher:
        if char in string.ascii_lowercase:
            count_list[lower_dict[char]] += 1
    
    sum_count = sum(count_list)
    count_list += count_list # repeat the list to allow access
    se_list = [0] * 26
    for i in range(1, 26): # iterate all offsets
        shifted_count_list = [count_list[k] for k in range(i, 26 + i)] # count_list decoded with current offset
        freq_list = [shifted_count_list[m]/sum_count for m in range(26)] # frequency of each letter in current string
        se_list[i] = sum([abs(freq_list[n]**2 - freq[list(freq.keys())[n]]**2) for n in range(26)])
    optimal_offset = np.argsort(np.array(se_list))[1]
    optimal_str = decode(input_cipher, optimal_offset)
    return optimal_str, optimal_offset

In [84]:
import random as rand

rhyme = """Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;
One for the master,
And one for the dame,
And one for the little boy
Who lives down the lane."""

encoded = encode(rhyme, rand.randint(10,20))
origin_str, opt_offset = auto_decode(encoded)
print(origin_str)
print(opt_offset)

baa, baa, black sheep,
have you any wool?
yes, sir, yes, sir,
three bags full;
one for the master,
and one for the dame,
and one for the little boy
who lives down the lane.
19
